# VaR & CVaR project 

In [34]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import norm, t
import matplotlib.pyplot as plt

### Data Preparation

In [35]:
def fetch_returns(ticker='SPY', start_date='2020-01-01', end_date='2024-12-31'):
    """Fetch historical returns for S&P 500 ETF"""
    data = yf.download(ticker, start=start_date, end=end_date)
    returns = data['Adj Close'].pct_change().dropna()
    return returns.squeeze()

sp500_returns = fetch_returns()
portfolio_value = 1e6 

[*********************100%***********************]  1 of 1 completed


### VaR Calculation

In [36]:
def calculate_var(returns, confidence_level=0.95, method='historical'):
    """Calculate Value-at-Risk using different methods"""
    if method == 'historical':
        return np.percentile(returns, 100*(1-confidence_level))
    
    elif method == 'normal':
        mu = np.mean(returns)
        sigma = np.std(returns)
        return norm.ppf(1-confidence_level, mu, sigma)
    
    elif method == 't-dist':
        params = t.fit(returns)
        return t.ppf(1-confidence_level, *params)
    
    else:
        raise ValueError("Invalid method. Choose: historical/normal/t-dist")

# Calculate 95% VaR using all methods
historical_var = calculate_var(sp500_returns, method='historical')
normal_var = calculate_var(sp500_returns, method='normal')
t_dist_var = calculate_var(sp500_returns, method='t-dist')

### CVaR Calculation

In [37]:
def calculate_cvar(returns, confidence_level=0.95):
    """Calculate Conditional Value-at-Risk"""
    var = calculate_var(returns, confidence_level, 'historical')
    tail_losses = returns[returns <= var]
    return np.mean(tail_losses)

cvar = calculate_cvar(sp500_returns)

### Backtesting

In [38]:
def kupiec_test(returns, var, confidence_level=0.99):
    """Kupiec's Proportion of Failures Test"""
    failures = returns < var
    p_actual = failures.mean()
    p_expected = 1 - confidence_level
    
    likelihood_ratio = -2 * (np.log((1-p_actual)**(len(returns)-failures.sum()) * p_actual**failures.sum()) 
                           - np.log((1-p_expected)**(len(returns)-failures.sum()) * p_expected**failures.sum()))
    
    return likelihood_ratio < 5.024  # Chi-square critical value (0.05, df=1)

backtest_result = kupiec_test(sp500_returns, historical_var)


### Visualization

In [39]:
def plot_risk_distribution(returns, var, cvar):
    plt.figure(figsize=(10,6))
    plt.hist(returns, bins=50, density=True, alpha=0.6, label='Daily Returns')
    
    plt.axvline(var, color='r', linestyle='--', 
                label=f'95% VaR ({var*100:.2f}%)')
    plt.axvline(cvar, color='darkred', linestyle='--',
                label=f'CVaR ({cvar*100:.2f}%)')
    
    plt.title('Portfolio Risk Distribution')
    plt.xlabel('Daily Returns')
    plt.ylabel('Frequency')
    plt.legend()
    plt.savefig('risk_distribution.png', dpi=300)
    plt.close()

plot_risk_distribution(sp500_returns, historical_var, cvar)

### Monte Carlo VaR

In [40]:
def monte_carlo_var(returns, days=1, simulations=10000):
    """Monte Carlo simulation for multi-day VaR"""

    if isinstance(returns, (pd.Series, pd.DataFrame)):
        returns = returns.values.squeeze() 
    
    mu = np.mean(returns)
    sigma = np.std(returns)
    drift = mu - 0.5*sigma**2
    
    random_shocks = np.random.normal(size=(days, simulations))
    daily_returns = np.exp(drift + sigma * random_shocks)
    
    portfolio_path = portfolio_value * daily_returns.prod(axis=0)
    
    sorted_returns = np.sort(portfolio_path)
    var_index = int(0.05 * simulations)
    return portfolio_value - sorted_returns[var_index]

sp500_returns = fetch_returns()
mc_var = monte_carlo_var(sp500_returns)


[*********************100%***********************]  1 of 1 completed


### Output

In [41]:
print(f"Historical VaR (95%): {historical_var*100:.2f}%")
print(f"Normal Dist VaR (95%): {normal_var*100:.2f}%") 
print(f"t-Dist VaR (95%): {t_dist_var*100:.2f}%")
print(f"CVaR (95%): {cvar*100:.2f}%")
print(f"Kupiec Test Passed: {backtest_result}")
print(f"Monte Carlo 1-day VaR: ${mc_var:,.2f}")

Historical VaR (95%): -1.93%
Normal Dist VaR (95%): -2.11%
t-Dist VaR (95%): -1.78%
CVaR (95%): -3.20%
Kupiec Test Passed: True
Monte Carlo 1-day VaR: $21,099.65
